# <h1><center>Workflow R</center></h1>

# Processamento das leituras
### Diretório de trabalho
Especifique a rota da pasta onde você fará suas análises

In [ ]:
setwd("~/Desktop/its

### Pacotes
Chame as librarias dos pacotes previamente isnstalados no R

In [ ]:
library("dada2")
packageVersion("dada2")
library("ShortRead")
packageVersion("ShortRead")
library("Biostrings")
packageVersion("Biostrings")
library("ngsReports")
library("dplyr")
library("pander")
library("fastqcr")

## Listando as sequências na pasta
Especifique a rota onde estão as sequências que serão análisadas

In [ ]:
dir_seqs = file.path("~/Desktop/its_invasoras")
list.files(dir_seqs)

## Padrão de reconhecimento 
Crie um padrão de reconhecimento para orientação das fitas forward e reverse

In [ ]:
fita_f = sort(list.files(dir_seqs, pattern ="R1_001.fastq.gz", full.names = TRUE))
fita_r = sort(list.files(dir_seqs, pattern ="R2_001.fastq.gz", full.names = TRUE))

## Identifcando os primers nas leituras
Neste caso para o experimento foram utilizados os seguintes primers:
- FWD ITS_86$\;\;\;$5′–GTGAATCATCGAATCTTTGAA–3′
- REV ITS_4R$\;\;\;$5′–TCCTCCGCTTATTGATATGC–3′

In [ ]:
FWD = "GTGAATCATCGAATCTTTGAA"
REV = "TCCTCCGCTTATTGATATGC"

- A seguinte função cria o complemento reverso da sequências dos primers especificados anteriormente

In [ ]:
orient_primer <- function(primer) {
   
    require(Biostrings)
    dna <- DNAString(primer) 
    orients <- c(Forward = dna, Complement = complement(dna), Reverse = reverse(dna), 
        RevComp = reverseComplement(dna))
    return(sapply(orients, toString))  
}
FWD.orients <- orient_primer(FWD)
REV.orients <- orient_primer(REV)
FWD.orients

- Agora será feito a remoção de bases ambiguas do tipo "N"

In [ ]:
fita_f.filtN <- file.path(dir_seqs, "filtN", basename(fita_f))
fita_r.filtN <- file.path(dir_seqs, "filtN", basename(fita_r))
filterAndTrim(fita_f, fita_f.filtN, fita_r, fita_r.filtN, maxN = 0, multithread = TRUE)

- A seguinte função conta o numero de vezes que os primers foram encontrados nas leituras contrarias

In [ ]:
primerHits <- function(primer, fn) {
   
    nhits <- vcountPattern(primer, sread(readFastq(fn)), fixed = FALSE)
    return(sum(nhits > 0))
}
rbind(Leitura_F = sapply(FWD.orients, primerHits, fn = fita_f.filtN[[1]]), 
    Leitura_R = sapply(FWD.orients, primerHits, fn = fita_r.filtN[[1]]), 
    Leitura_F = sapply(REV.orients, primerHits, fn = fita_f.filtN[[1]]), 
    Leitura_R = sapply(REV.orients, primerHits, fn = fita_r.filtN[[1]]))

## Removendo os primers das sequências
- Instale primeiro cutadapt via terminal com seguinte comando
<br>
"conda install -c bioconda cutadapt"
<br>
- Especifique a rota de cutadapt para usar a ferramenta na interface do R

In [ ]:
cutadapt <- "/home/alejandro/anaconda3/bin/cutadapt"
system2(cutadapt, args = "--version")

- criando a função para cortar os primers com cutadapt

In [ ]:
dir_seqs.cut <- file.path(dir_seqs, "cutadapt")
if(!dir.exists(dir_seqs.cut)) dir.create(dir_seqs.cut)
fita_f.cut <- file.path(dir_seqs.cut, basename(fita_f))
fita_r.cut <- file.path(dir_seqs.cut, basename(fita_r))

FWD.RC <- dada2:::rc(FWD)
REV.RC <- dada2:::rc(REV)

R1.flags <- paste("-g", FWD, "-a", REV.RC) 

R2.flags <- paste("-G", REV, "-A", FWD.RC) 

for(i in seq_along(fita_f)) {
  system2(cutadapt, args = c(R1.flags, R2.flags, "-n", 2,
                             "-o", fita_f.cut[i], "-p", fita_r.cut[i], 
                             fita_f.filtN[i], fita_r.filtN[i])) 
}


- checando se efitavamente os primers foram removidos das sequências

In [ ]:
rbind(Leitura_F = sapply(FWD.orients, primerHits, fn = fita_f.cut[[1]]), 
    Leitura_R = sapply(FWD.orients, primerHits, fn = fita_r.cut[[1]]), 
    Leitura_F = sapply(REV.orients, primerHits, fn = fita_f.cut[[1]]), 
    Leitura_R = sapply(REV.orients, primerHits, fn = fita_r.cut[[1]]))

- Manipulação strig para extrair o nome das amostras

In [ ]:
cutFs <- sort(list.files(dir_seqs.cut, pattern = "R1_001.fastq.gz", full.names = TRUE))
cutRs <- sort(list.files(dir_seqs.cut, pattern = "R2_001.fastq.gz", full.names = TRUE))

In [ ]:
get.sample.name <- function(fname) strsplit(basename(fname), "_")[[1]][1]
sample.names <- unname(sapply(cutFs, get.sample.name))
head(sample.names)

- Criando objetos para as fitas Forward e Reverse

In [ ]:
filtFs <- file.path(dir_seqs.cut, "filtered", basename(cutFs))
filtRs <- file.path(dir_seqs.cut, "filtered", basename(cutRs))

# <h1><center> Inspeção de qualidade das librarias sequenciadas após filtragem</center></h1>

- Aplicando filtros às leituras

In [ ]:
seqs_fltrds <- filterAndTrim(cutFs, filtFs, cutRs, filtRs, maxN = 0, maxEE = c(2, 2), 
    truncQ = 2, minLen = 100, minQ= c(3,3), rm.phix = TRUE, compress = TRUE, multithread = TRUE)  
head(seqs_fltrds)

- inspeccionando as sequencias após filtragem

In [ ]:
fastqc(fq.dir = "~/Desktop/its_invasoras/cutadapt/filtered", qc.dir = "~/Desktop/its_invasoras/cutadapt/filtered/fastqc_rsltds", threads = 4, fastqc.path = "~/bin/FastQC/fastqc")

In [ ]:
seqs.dir = file.path("~/Desktop/its_invasoras/cutadapt/filtered")
seqs.list = list.files(seqs.dir, pattern = "fastqc.zip$", full.names = TRUE)
seqs.list



In [ ]:
seqs.report = FastqcDataList(seqs.list)
leituras <- readTotals(seqs.report)
leituras %>%
    dplyr::filter(grepl("R1_001", Filename)) %>% 
    pander(
        big.mark = ",",
        caption = "Total de Leituras p/fita forward", 
        justify = "lr"
    )

In [ ]:
getModule(seqs.list[1], "Sequence_Length_Distribution")

In [ ]:
plotBaseQuals(seqs.list[1], plotType = "boxplot" , usePlotly = F, boxWidth = 0.7)

# <h1><center> Construindo o modelo de de erros</center></h1>

In [ ]:
errF <- learnErrors(filtFs, multithread = TRUE)

In [ ]:
plotErrors(errF, nominalQ = TRUE)

## Derreplicando sequencias

In [ ]:
derepFs <- derepFastq(filtFs, verbose = TRUE)
# Name the derep-class objects by the sample names
names(derepFs) <- sample.names


## Inferência das amostras

In [ ]:
dadaFs <- dada(derepFs, err = errF, multithread = TRUE)

## Construindo a tabela

In [ ]:
seqtab <- makeSequenceTable(dadaFs)
dim(seqtab)

## Removendo as qimeras

In [ ]:
seqtab.nochim <- removeBimeraDenovo(seqtab, method="consensus", multithread=TRUE, verbose=TRUE)

- olhando a distribuição dos tamanhos das sequências

In [ ]:
table(nchar(getSequences(seqtab.nochim)))

## Estatisticas gerais

In [ ]:
getN <- function(x) sum(getUniques(x))
track <- cbind(seqs_fltrds, sapply(dadaFs, getN), rowSums(seqtab.nochim))

colnames(track) <- c("Seqs/entrada", "Seqs/filtradas", "SeqsF/denoised","Seqs/não quimeras")
rownames(track) <- sample.names
head(track)

# <h1><center>Workflow QIIME 2</center></h1>

## Exportando a tabela para QIIME 2

- Tabela de frequência

In [ ]:
write.table(t(seqtab.nochim), "dada2_analysis_seqtab_nochim.txt", sep="\t", row.names=TRUE, col.names=NA, quote=FALSE)

- Sequências

In [ ]:
uniquesToFasta(seqtab.nochim, fout='dada2_analysis_rep_seqs.fna', ids=colnames(seqtab.nochim))

## Importando a tabela e sequências para QIIME2
### Comandos para QIIME2 comand interface

- Fazendo a conversão da tabela em formato .biom

In [ ]:
echo -n "#AVS Table" | cat - dada2_analysis_seqtab_nochim.txt > dada2_analysis_biom_table.txt

In [ ]:
biom convert -i dada2_analysis_biom_table.txt -o dada2_analysis_table.biom --table-type="OTU table" --to-hdf5

### Importando a tabela em um artefato .qza

In [ ]:
qiime tools import \
--input-path dada2_analysis_table.biom \
--type 'FeatureTable[Frequency]' \
--input-format BIOMV210Format \
--output-path table.qza


### Importando as sequências processadas em um artefato .qza

In [ ]:
qiime tools import \
--input-path dada2_analysis_rep_seqs.fna \
--type 'FeatureData[Sequence]' \
--output-path rep-seqs.qza

### Importando a base de dados de referência UNITE (99) para classificação taxonômica das sequências

- Importando as sequências de referência em um artefato .qza

In [ ]:
qiime tools import \
  --type 'FeatureData[Sequence]' \
  --input-path sh_refs_qiime_ver8_99_s_02.02.2019.fasta \
  --output-path unite_seqs_99.qza

- Importando as etiquetas taxonômicas das sequências de referência em um artefato .qza

In [ ]:
qiime tools import \
  --type 'FeatureData[Taxonomy]' \
  --input-format HeaderlessTSVTaxonomyFormat \
  --input-path sh_taxonomy_qiime_ver8_99_s_02.02.2019.txt \
  --output-path ref_taxonomy_unite_99.qza

### Criando e treinando o classificador em CYVERSE

- Importando artefatos de sequências de referência e taxonômia para treinar o classificador 

In [ ]:
!qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads unite_seqs_99.qza \
  --i-reference-taxonomy ref_taxonomy_unite_99.qza \
  --o-classifier classifier_unite_99.qza

- Classificando as sequências do experimento

In [ ]:
!qiime feature-classifier classify-sklearn \
  --i-classifier classifier_unite_99.qza \
  --i-reads rep-seqs.qza \
  --o-classification taxonomy.qza

## Gráfico barplot de abundância relativa

In [ ]:
!qiime taxa barplot \
  --i-table table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file sample-metadata.tsv \
  --o-visualization taxa-bar-plots.qzv